Code is from:
https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
#WRITE TOKEN HERE
os.environ['TOKEN'] = ''

NameError: name 'os' is not defined

In [3]:
def auth():
    return os.getenv('TOKEN')

In [4]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [17]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #CHANGE THIS PART TO DISPLAY DIFFERENT VALUES
    query_params = {'query': keyword,
                    'max_results': max_results,
                    #'expansions': , #'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'text,public_metrics,referenced_tweets,geo,created_at', #'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'name,username,created_at,description,public_metrics,verified', #'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,country,country_code,geo,name,place_type', #'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [18]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [19]:
#CHANGE
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "UCLA"
start_time = "2021-03-01T00:00:00.000Z"
end_time = "2021-03-31T00:00:00.000Z"
max_results = 15

In [20]:
url = create_url(keyword, start_time,end_time, max_results)
print(url)
json_response = connect_to_endpoint(url[0], headers, url[1])

('https://api.twitter.com/2/tweets/search/recent', {'query': 'UCLA', 'max_results': 15, 'tweet.fields': 'text,public_metrics,referenced_tweets,geo,created_at', 'user.fields': 'name,username,created_at,description,public_metrics,verified', 'place.fields': 'full_name,country,country_code,geo,name,place_type', 'next_token': {}})
Endpoint Response Code: 200


In [21]:
print(json.dumps(json_response, indent=4, sort_keys=True))

{
    "data": [
        {
            "created_at": "2022-03-02T06:46:50.000Z",
            "id": "1498912699446554624",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 0
            },
            "referenced_tweets": [
                {
                    "id": "1498891622699847684",
                    "type": "replied_to"
                }
            ],
            "text": "@zagafan2 @tucsontwitchy @glizzert00 @SethDavisHoops The win at ucla is impressive for sure. By 20 is a whoopin"
        },
        {
            "created_at": "2022-03-02T06:45:57.000Z",
            "id": "1498912474472583174",
            "public_metrics": {
                "like_count": 0,
                "quote_count": 0,
                "reply_count": 0,
                "retweet_count": 74
            },
            "referenced_tweets": [
                {
                    "id": "149882

In [22]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
#csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

**The part below is to make the csv file, but I still need to fix it**

In [23]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        print(tweet)
    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter)

In [ ]:

        #'text,public_metrics,referenced_tweets,geo,created_at', #'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
         #           'user.fields': 'name,username,created_at,description,public_metrics,verified', #'id,name,username,created_at,description,public_metrics,verified',
          #          'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        #author_id = tweet['author_id']

        # 2. Time created
        #created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        if ('geo' in tweet):   
            geo = tweet['geo']['place_id']
        else:
            geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        #lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        #source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [ geo, tweet_id,  like_count, quote_count, reply_count, retweet_count, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1


In [26]:
print(json_response)

{'data': [{'referenced_tweets': [{'type': 'replied_to', 'id': '1498891622699847684'}], 'created_at': '2022-03-02T06:46:50.000Z', 'id': '1498912699446554624', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': '@zagafan2 @tucsontwitchy @glizzert00 @SethDavisHoops The win at ucla is impressive for sure. By 20 is a whoopin'}, {'referenced_tweets': [{'type': 'retweeted', 'id': '1498823653022126084'}], 'created_at': '2022-03-02T06:45:57.000Z', 'id': '1498912474472583174', 'public_metrics': {'retweet_count': 74, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': 'RT @ComplexSneakers: Air Jordan 6 “UCLA” PE 👀 [🎥: @ginamconti] https://t.co/pnfSMuz04U'}, {'created_at': '2022-03-02T06:45:41.000Z', 'id': '1498912408613826562', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': 'Go Toトラベル利用者の方が新型コロナウイルス感染症を示唆する症状をより多く経験していることが明らかに⇒「不正確」\nSFSSが東京大学／UCLAの疫学調査研究をファクトチェック！\n#ファクトチェック\nhttps:/

In [24]:
append_to_csv(json_response, "data.csv")

{'referenced_tweets': [{'type': 'replied_to', 'id': '1498891622699847684'}], 'created_at': '2022-03-02T06:46:50.000Z', 'id': '1498912699446554624', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': '@zagafan2 @tucsontwitchy @glizzert00 @SethDavisHoops The win at ucla is impressive for sure. By 20 is a whoopin'}
{'referenced_tweets': [{'type': 'retweeted', 'id': '1498823653022126084'}], 'created_at': '2022-03-02T06:45:57.000Z', 'id': '1498912474472583174', 'public_metrics': {'retweet_count': 74, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': 'RT @ComplexSneakers: Air Jordan 6 “UCLA” PE 👀 [🎥: @ginamconti] https://t.co/pnfSMuz04U'}
{'created_at': '2022-03-02T06:45:41.000Z', 'id': '1498912408613826562', 'public_metrics': {'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}, 'text': 'Go Toトラベル利用者の方が新型コロナウイルス感染症を示唆する症状をより多く経験していることが明らかに⇒「不正確」\nSFSSが東京大学／UCLAの疫学調査研究をファクトチェック！\n#ファクトチェック\nhttps://t.co/a8Coon

In [25]:
import pandas as pd
df = pd.read_csv("data.csv")
df

,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
0,author id,created_at,geo,id,lang,like_count,quote_count,reply_count,retweet_count,source,tweet
1,,1498910344315424768,0,0,0,84,RT @UCLAFootball: Make that 3️⃣7️⃣ unanswered ...,NaN,NaN,NaN,NaN
2,,1498910226753462272,0,0,0,72,RT @ComplexSneakers: Air Jordan 6 “UCLA” PE 👀 ...,NaN,NaN,NaN,NaN
3,,1498909950726320133,0,0,0,16,RT @schadenfraade: The Bel Air HOA is treating...,NaN,NaN,NaN,NaN
4,,1498909543715254279,0,0,1,0,"@pgz26 If it don't apply, let it fly...plus Lo...",NaN,NaN,NaN,NaN
5,,1498908653272776704,1,0,0,0,The NCAA tournament field better hope they cat...,NaN,NaN,NaN,NaN
6,,1498908637397336067,0,0,0,0,@BookingwWylie Fucking UCLA hit too. Texas Tec...,NaN,NaN,NaN,NaN
7,,1498908561954336769,1,0,0,0,@RyperiousPeople @equitybruin Can’t happen any...,NaN,NaN,NaN,NaN
8,,1498908520338526210,0,0,0,0,@Dawgman6_7 @JaedenJMoore @adamgorney @Mason_H...,NaN,NaN,NaN,NaN
9,,1498908170835542017,0,0,0,0,Hey @UCLA— your parking enforcement sucks.,NaN,NaN,NaN,NaN
